# Spotify Feature Engineering

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

NUM_CLUSTERS = 7

In [ ]:
df = pd.read_csv("../data/tops/features.csv", index_col=0)

## Function Schema

Within the Notebook, we have the following functions to assist our feature engineering and clustering:

>1. `clean_raw_df`: Returns a clean version of the raw DataFrame.
2. `one_hot`: One hot encodes the Mode and Time Signature features.
3. `center`: Centers the data of a DataFrame by subtracting the mean row.
4. `design_matrix`: Returns a design matrix (NumPy) for future work.
5. `plot_scree`: Given a SVD of a matrix, plots the scree plot.
6. `cluster_data`: Performs k-Means clustering on the given data.
7. `plot_pca`: Plots the dimensionality-reduced data using first two principal components.
8. `pipeline`: Performs the whole process.
>

In [ ]:
def clean_raw_df(df: pd.DataFrame, pipe=True) -> pd.DataFrame:
    """Returns a clean version of the raw DataFrame"""
    df = df.drop_duplicates(subset=["uri"]).set_index("uri").dropna()
    df = df[~df['title'].str.contains("rain", case=False)]
    if pipe:
        df = df.drop(["track_href", "id", "title", "type", "analysis_url"], axis=1)
    df = df[df['time_signature'] > 0.0]
    df = df.replace({"mode": {1.0:"major", 0.0: "minor"}})

    return df

In [ ]:
def one_hot(df: pd.DataFrame) -> pd.DataFrame:
    """One hot encodes the Mode and Time Signature Features"""
    oh_enc = OneHotEncoder()
    oh_enc.fit(df[['mode', "time_signature", "key"]])
    dummies = pd.DataFrame(oh_enc.transform(df[['mode', "time_signature", "key"]]).todense(), 
                           columns=oh_enc.get_feature_names(['mode', "time_signature", "key"]),
                           index = df.index)
    return df.join(dummies).drop(['mode', "time_signature", "key"], axis=1)

In [ ]:
def center(df: pd.DataFrame) -> pd.DataFrame:
    """ Returns a design matrix for PCA from a given DataFrame"""
    means = np.mean(df)
    centered = df - means
    
    return centered

In [ ]:
def design_matrix(df: pd.DataFrame) -> pd.DataFrame:
    """Returns the design matric of the DataFrame"""
    return StandardScaler().fit_transform(df.to_numpy())

In [ ]:
def plot_scree(df: pd.DataFrame) -> None:
    """Creates a scree plot of the singular values of the SVD"""
    u, s, vt = np.linalg.svd(design_matrix(df), full_matrices=False)
    num_features = s.shape[0] + 1
    
    plt.figure(figsize=(10,5))
    plt.plot(list(range(1,num_features)), s**2 / sum(s**2))
    plt.xticks(list(range(1, num_features)), list(range(1,num_features)))
    plt.xlabel('PC #');
    plt.ylabel('Fraction of Variance Explained');
    plt.title('Fraction of Variance Explained by each Principal Component');

In [ ]:
def cluster_data(df: pd.DataFrame, clusters=NUM_CLUSTERS) -> tuple:
    """Performs the KMeans Clustering of the data"""
    X = design_matrix(df)
    pca = PCA(n_components=2)
    principal_components = pca.fit_transform(X)

    target_df_pca = pd.DataFrame(data=principal_components, columns=['PC_1', 'PC_2'])

    kmeans_pca = KMeans(n_clusters=clusters, init='k-means++', n_init=25)
    predicted_pca = kmeans_pca.fit_predict(principal_components)
    
    target_df_pca['prediction'] = predicted_pca
    target_df_pca.set_index(keys=df.index, inplace=True)
    
    return target_df_pca, kmeans_pca

In [ ]:
def plot_pca(cluster_data: tuple) -> None:
    """Plot a PCA chart with n clusters"""
    data, kmeans = cluster_data
    n = kmeans.n_clusters
    plt.figure(figsize=(10,5))
    sns.scatterplot(x='PC_1', y='PC_2', data=data, 
                    hue='prediction', palette=sns.color_palette('hls', n), 
                    style='prediction', markers=n*['.'], s=100)

    for center, i in zip(kmeans.cluster_centers_, range(n)):
        if not i:
            plt.scatter(center[0], center[1], marker='*',s=200, c='black', label='Centroid')
        else:
            plt.scatter(center[0], center[1], marker='*',s=200, c='black')

    plt.xlabel('First Principal Component (PCA)')
    plt.ylabel('Second Principal Component (PCA)')
    plt.title('k-Means Clustering via PCA', pad=20)
    plt.legend(shadow=True);

In [ ]:
def pipeline(df, plot=True, return_data=False):
    """Completes a full cycle of the PCA, Clustering processs for a given DataFrame."""
    cleaned = clean_raw_df(df)
    oh = one_hot(cleaned)
    centered = center(oh)
    cluster = cluster_data(centered)
    
    if plot:
        plot_scree(centered)
        plot_pca(cluster)
    
    if return_data:
        return cluster[0], cluster[1], centered

## PCA --> k-Means Clustering

Within this section of the notebook, I will perform Principal Component Analysis (PCA) on the featured data, and then use a k-Means Clustering algorithm to identify genres in my two-dimensional track space.

In [ ]:
data, kmeans, clustered = pipeline(df, plot=True, return_data=True)

In [ ]:
preds = data.merge(clean_raw_df(df, pipe=False), how="inner", 
                left_on=data.index, 
                right_on=clean_raw_df(df, pipe=False).index, 
                left_index=True, 
                right_index=True)

In [ ]:
fig = px.scatter(preds, x='PC_1', y='PC_2', hover_name='title', color="prediction")
fig.update_layout(
    height=600,
    width=900,
    title_text='PCA of Spotify Tracks'
)
fig.show()

## k-Means Clustering --> PCA

Within this section of the notebook, I will use a k-Means Clustering algorithm to identify genres in my higher-dimensional track space, and then perform Principal Component Analysis (PCA) to reduce the featured data into a visualizable, two-dimensional space.

In [ ]:
new_df = center(one_hot(clean_raw_df(df)))
X = design_matrix(new_df)

model = KMeans(n_clusters=NUM_CLUSTERS, init='k-means++', n_init=25)
pca = PCA(n_components=2)

predictions = model.fit_predict(X)
principal_components = pca.fit_transform(X)

target_df_pca = pd.DataFrame(data=principal_components, columns=['PC_1', 'PC_2'], index=new_df.index)
joined_df = new_df.join(target_df_pca)

joined_df['predictions'] = predictions.astype(str)
joined_df['title'] = clean_raw_df(df, pipe=False)['title']

In [ ]:
plt.figure(figsize=(20,10))
sns.scatterplot(x='PC_1', y='PC_2', data=joined_df, 
                hue='predictions', palette=sns.color_palette('hls', NUM_CLUSTERS), 
                style='predictions', markers=NUM_CLUSTERS*['.'], s=100)

for centroid, i in zip(model.cluster_centers_, range(NUM_CLUSTERS)):
    if not i:
        plt.scatter(centroid[0], centroid[1], marker='*',s=200, c='black', label='Centroid')
    else:
        plt.scatter(centroid[0], centroid[1], marker='*',s=200, c='black')

plt.xlabel('First Principal Component (PCA)')
plt.ylabel('Second Principal Component (PCA)')
plt.title('k-Means Clustering via PCA', pad=20)
plt.legend(shadow=True);

In [ ]:
fig2 = px.scatter(joined_df, x='PC_1', y='PC_2', hover_name='title', color="predictions")
fig2.update_layout(
    height=900,
    width=1200,
    title_text='PCA of Spotify Tracks'
)
fig2.show()